In [ ]:
## Set notebook to auto reload updated modules
%load_ext autoreload
%autoreload 2

In [ ]:
%env ENV_FOR_DYNACONF=dev

In [ ]:
from owm_bot.core.config import (
    AppSettings,
    settings,
    OpenweathermapSettings,
    owm_settings,
)

In [ ]:
LOGGING_LEVEL = settings.log_level

In [ ]:
import typing
from pathlib import Path
import json
import random

import logging
import logging.config

In [ ]:
import red_logging

_formatter = red_logging.get_formatter_config(
    name="nb",
    fmt="%(asctime)s | %(levelname)s | %(message)s",
    datefmt=red_logging.fmts.DATE_FMT_DATE_ONLY,
)
console_handler = red_logging.get_streamhandler_config(
    name="console", formatter="nb", level="WARNING"
)
_handler = red_logging.get_streamhandler_config(
    name="console_nb", formatter="nb", level="DEBUG"
)
_logger = red_logging.get_logger_config(
    name="nb", handlers=["console_nb"], level=LOGGING_LEVEL
)

In [ ]:
logging_config = red_logging.assemble_configdict(
    disable_existing_loggers=True,
    formatters=[_formatter],
    handlers=[_handler, console_handler],
    loggers=[_logger],
)

In [ ]:
logging.config.dictConfig(logging_config)
log = logging.getLogger("nb")

In [ ]:
from owm_bot.core import (
    DATA_DIR,
    CACHE_DIR,
    SERIALIZE_DIR,
    PQ_DIR,
    ENSURE_DIRS,
    OUTPUT_DIR,
)
from owm_bot.core import (
    PQ_ENGINE,
    OPENWEATHERMAP_BASE_URL,
    OPENWEATHERMAP_ONECALL_URL,
    OPENWEATHERMAP_GEO_URL,
)
from owm_bot.setup import setup_dirs
from owm_bot.domain.Location import (
    JsonLocation,
    JsonLocationsLoader,
    OwmGeoLookup,
)

from owm_bot.domain.Weather.current import OWMCurrentWeather
from owm_bot.domain.Weather.forecast import (
    OWMForecastWeather,
    OWMForecastWeatherEntry,
)
from owm_bot.utils import data_utils
from owm_bot.controllers import OpenWeathermapController
from owm_bot.weather import (
    CurrentWeatherPQFileController,
    ForecastWeatherPQFileController,
)

In [ ]:
import pandas as pd
from red_utils.ext.dataframe_utils import pandas_utils

pd.set_option("display.max_columns", None)

In [ ]:
setup_dirs(ensure_dirs=ENSURE_DIRS)

In [ ]:
GET_WEATHER: bool = False

---

In [ ]:
if GET_WEATHER:
    with OpenWeathermapController(units=owm_settings.units) as owm_ctl:
        current_weather: OWMCurrentWeather = owm_ctl.current_weather(save_pq=True)
        # log.debug(f"Current weather: {current_weather}")
        log.info("Refreshed current weather")

        weather_forecast: OWMForecastWeather = owm_ctl.weather_forecast(save_pq=True)
        # log.debug(f"Weather forecast: {weather_forecast}")
        log.info("Refreshed weather forecast")

In [ ]:
with CurrentWeatherPQFileController() as currentweather_ctl:
    current_weather_df = currentweather_ctl.df

In [ ]:
current_weather_df.head(5)

In [ ]:
with ForecastWeatherPQFileController() as weatherforecast_ctl:
    weather_forecast_df = weatherforecast_ctl.df

In [ ]:
weather_forecast_df.head(5)